In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pickle

ModuleNotFoundError: No module named 'requests'

In [24]:
def get_ids(url):
    '''
    Given a url, returns a dictionary of pdf ids and their corresponding urls.
    '''
    try:
        response = requests.get(url)
        html = response.text

        # find all strings of the form "id='num-of-ratings-[0-9]+'" under the "Scores" section and extract the number
        html = html[html.find('id="Sheet_Music"'):]


        pdf_id = re.findall(r'id=\'num-of-ratings-[0-9]+\'', html)
        pdf_id = [int(re.findall(r'[0-9]+', r)[0]) for r in pdf_id]
        pdf_dict = {}
        for i in pdf_id:
            pdf_dict[i] = url
        return pdf_dict
    except:
        return {}

def get_multiple_ids(urls):
    '''
    Given a list of urls, returns a dictionary of pdf ids and their corresponding urls.
    '''
    pdf_dict = {}
    for url in urls:
        pdf_dict.update(get_ids(url))
    return pdf_dict

In [17]:
# get links in 9 way dataset
with open('9_way_list-Copy1.txt', 'r') as file:
    composers = file.readlines()

composers = ['https://imslp.org/wiki/Category:' + composer[:-1] for composer in composers]
print(composers)

['https://imslp.org/wiki/Category:Bach,_Johann_Sebastian', 'https://imslp.org/wiki/Category:Beethoven,_Ludwig_van', 'https://imslp.org/wiki/Category:Chopin,_Fr%C3%A9d%C3%A9ric', 'https://imslp.org/wiki/Category:Haydn,_Joseph', 'https://imslp.org/wiki/Category:Liszt,_Franz', 'https://imslp.org/wiki/Category:Mozart,_Wolfgang_Amadeus', 'https://imslp.org/wiki/Category:Schubert,_Franz', 'https://imslp.org/wiki/Category:Schumann,_Robert', 'https://imslp.org/wiki/Category:Scriabin,_Aleksand']


In [19]:
# code the previous team uses to find all the urls for a composer

def find_score_url(url):
    """
    This function takes in one arguments:
    1) url, a composer url page to parse
    This function returns a list of piece url for a specific composer.
    """
    list_page = [url]
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    # add everything on the first page to the list
    result = soup.find_all('div', class_="jq-ui-tabs" )
    if result == []:
        return "badlink"
    scores = result[0].find_all('a',class_='categorypagelink')
    url_list = [link.get('href') for link in scores]
    # check sub-page
    list_page = find_next_score_url(url)
    if list_page != []:
        for page in list_page:
            r = requests.get(page)
            soup = BeautifulSoup(r.text, "html.parser")
            url_list += [link.get('href') for link in soup.find_all('a',class_='categorypagelink')]
    return url_list

def find_next_score_url(url):
    """
    Helper function for find_score_url
    """
    result = []
    while True:
        r = requests.get(url)
        soup = BeautifulSoup(r.text, "html.parser")
        url_list = [link.get('href') for link in soup.find_all('a',class_='categorypaginglink',string='next 200')]
        if url_list == []:
            break
        url = 'http://imslp.org' + url_list[-1]
        result.append(url)
    return result

urls = []
for composer in composers:
    urls += find_score_url(composer)

urls= ['https://imslp.org' + s for s in urls]

In [35]:
# get all the pdf ids from the urls
pdf_dict = get_multiple_ids(urls)

ConnectionError: HTTPSConnectionPool(host='imslp.orgb', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x10e06ff90>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [34]:
print(pdf_dict)

dict_keys([466135, 466136, 466137, 772069, 772070, 772071, 772072, 837835, 772065, 772066, 772067, 772068, 772076, 772077, 772078, 772079, 837837, 837836, 466138, 466139, 466140, 461082, 461083, 461084, 461085, 311373, 772456, 772457, 772458, 772459, 772462, 772463, 772464, 772465, 837949, 772473, 772474, 772475, 772476, 837951, 837950, 466141, 466142, 466143, 461055, 461056, 461057, 461058, 461047, 773072, 773073, 773074, 773075, 773076, 773077, 773078, 773079, 838101, 259803, 259804, 259805, 259806, 773080, 773081, 773082, 773083, 838103, 838102, 466144, 466145, 466146, 773561, 773562, 773563, 773564, 773567, 773568, 773569, 773570, 838118, 773574, 773575, 773576, 773577, 838120, 838119, 466147, 466148, 466149, 774313, 774314, 774315, 774316, 774317, 774318, 774319, 774320, 774321, 774322, 774323, 774324, 838185, 838186, 838187, 466150, 466151, 466152, 461095, 461096, 461097, 461098, 461090, 774861, 774862, 774863, 774864, 774865, 774866, 774867, 774868, 774869, 774870, 774871, 77487

In [30]:
def save_list_to_file(lst, filename):
    with open(filename, 'w') as f:
        for item in lst:
            f.write(str(item) + '\n')
            
def save_dict_to_pickle(dictionary, filename):
    with open(filename, 'wb') as f:
        pickle.dump(dictionary, f)
        
# create file for all urls from 9-way composers
save_list_to_file(urls, '9_way_urls_for_all_scores.txt')
save_dict_to_pickle(pdf_dict, '9_way_pdf_dict.pkl')